In [33]:
import tensorflow as tf
import cv2
import numpy as np
import dlib
import pickle 
import matplotlib.pyplot as plt 

In [34]:
cnn_model = tf.keras.models.load_model('mask_model',compile=False)
scaler = pickle.load(open('scaler.pkl', 'rb'))
face_detector = dlib.get_frontal_face_detector()

In [35]:
def draw_mask(rgb,x,y,w,h,v=20,color=(255,0,0),thickness=2):
    cv2.line(rgb,(x,y),(x+v,y),color,thickness)
    cv2.line(rgb,(x,y),(x+v,y),color,thickness)

    cv2.line(rgb,(x+w,y),(x+w-v,y),color,thickness)
    cv2.line(rgb,(x+w,y),(x+w,y+v),color,thickness)

    cv2.line(rgb,(x,y+h),(x,y+h-v),color,thickness) 
    cv2.line(rgb,(x,y+h),(x+v,y+h),color,thickness)

    cv2.line(rgb,(x+w,y+h),(x+w,y+h-v),color,thickness)
    cv2.line(rgb,(x+w,y+h),(x+w-v,y+h),color,thickness)


In [36]:
cap = cv2.VideoCapture(0)
fit = 20
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nb_faces = face_detector(gray)
    if len(nb_faces)>0:
        for counter,face in enumerate(nb_faces):
            try : 
                x1,y1 = face.left(),face.top()
                x2,y2 = face.right(),face.bottom()

                x = x1-fit
                y = y1-fit 
                w = x2+fit 
                h = y2+fit 
                
            
                img_crop = gray[y:h,x:w]
                img_crop = np.array(img_crop)
                img_crop = cv2.resize(img_crop,(224,224))


                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.rectangle(frame,(x1,y1),(x2,y2),(220,255,220),1)

            
                X = np.array(img_crop)
                X = scaler.transform(X.reshape(1,224*224))
                X = X.reshape(-1,224,224,1)
                Z = cnn_model.predict(X)
                A = Z[0]

                if A[1]>A[0]:
                    cv2.putText(frame,f"Mask {round(A[1]* 100,2)} %",(10,450),font,1,thickness=3,color=(0,255,0))
                    draw_mask(frame,x1,y1,x2-x1,y2-y1,10,(0,255,0),3)
                else:
                    cv2.putText(frame,f"No Mask {round(A[0]*100,2)} %",(10,450),font,1,thickness=3,color=(0,0,255))
                    draw_mask(frame,x1,y1,x2-x1,y2-y1,10,(0,0,255),3)


            except Exception as e :
                print("Erreur")
    else :  
        font = cv2.FONT_HERSHEY_SIMPLEX

        X = cv2.resize(gray,(224,224))
        X = scaler.transform(X.reshape(1,224*224))
        X = X.reshape(-1,224,224,1)
        Z = cnn_model.predict(X)
        A = Z[0]

        if A[1]>A[0]:
            cv2.putText(frame,f"Mask {round(A[1]* 100,2)} %",(10,450),font,1,thickness=3,color=(0,255,0))
        else:
            cv2.putText(frame,f"No Mask {round(A[0]*100,2)} %",(10,450),font,1,thickness=3,color=(0,0,255))


    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
